# Competição ML @SBS/DAA - 4ª Edição (2021/2022)
Esta competição está relacionada com o Trabalho Prático de Grupo das UCs do perfil Machine Learning: Fundamentos e Aplicações da Uminho e da UC Dados e Aprendizagem Automática, tendo, como destinatários, alunos do Mestrado em Engenharia Informática, do Mestrado em Matemática e Computação, e do Mestrado em Engenharia de Sistemas.

# Descrição da Competição
A modelação do fluxo de tráfego rodoviário é um conhecido problema de características estocásticas, não-lineares. Tem, contudo, aparecido na literatura um conjunto de modelos que demonstram um potencial assinalável neste tipo de previsões. Com isso em consideração, foi construído um *dataset* que contém dados referentes ao tráfego de veículos na cidade do Porto durante um período superior a 1 ano. O *dataset* cobre um período que vai desde o dia 24 de julho de 2018 até ao dia 02 de outubro de 2019.

Com esta competição espera-se que os alunos desenvolvam e otimizem modelos de *Machine Learning* que sejam capazes de prever o fluxo de tráfego rodoviário, numa determinada hora, na cidade do Porto.

In [5]:
!pip install kaggle

In [6]:
#!kaggle competitions download -c tpsbsdaa2122

In [7]:
#import kaggle 
import pandas as pd
import seaborn as sns
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.tree import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

## Load do *dataset* 

In [8]:
training_file = '../input/transit/training_data.csv'
test_file = '../input/transit/test_data.csv'

df = pd.read_csv(training_file,encoding = "ISO-8859-1")
df_test = pd.read_csv(test_file,encoding = "ISO-8859-1")

In [10]:
df.head()

In [11]:
df.info()

In [12]:
df.describe()

Comparando os valores *max* de cada feature com os valores *mean* observamos uma grande discrepância entre o valor médio de *AVERAGE_TIME_DIFF* e o valor máximo desta *feature*. Pode ser que estejamos perante *outliers*.

In [13]:
df.columns

## Features
* **city_name** - nome da cidade em causa;
* **record_date** - o timestamp associado ao registo;
* **average_speed_diff** - a diferença de velocidade corresponde à diferença entre (1.) a velocidade máxima que os carros podem atingir em cenários sem trânsito e (2.) a velocidade que realmente se verifica. Quanto mais alto o valor, maior é a diferença entre o que se está a andar no momento e o que se deveria estar a andar sem trânsito, i.e., valores altos deste atributo implicam que se está a andar mais devagar;
* **average_free_flow_speed** - o valor médio da velocidade máxima que os carros podem atingir em cenários sem trânsito;
* **average_time_diff** - o valor médio da diferença do tempo que se demora a percorrer um determinado conjunto de ruas. Quanto mais alto o valor maior é a diferença entre o tempo que demora para se percorrer as ruas e o que se deveria demorar sem trânsito, i.e., valores altos implicam que se está a demorar mais tempo a atravessar o conjunto de ruas;
* **average_free_flow_time** - o valor médio do tempo que demora a percorrer um determinado conjunto de ruas quando não há trânsito;
* **luminosity** - o nível de luminosidade que se verificava na cidade do Porto;
* **average_temperature - o valor médio da temperatura para o record_date na cidade do Porto;
* **average_atmosp_pressure** - o valor médio da pressão atmosférica para o record_date;
* **average_humidity** - o valor médio da humidade para o record_date;
* **average_wind_speed** - o valor médio da velocidade do vento para o record_date;
* **average_cloudiness** - o valor médio da percentagem de nuvens para o record_date;
* **average_precipitation** - o valor médio de precipitação para o record_date;
* **average_rain** - avaliação qualitativa da precipitação para o record_date.

In [14]:
print(df.count())

In [15]:
print(df.isna().sum())

In [16]:
sns.heatmap(df.isnull(), yticklabels=False, cbar=True, cmap='viridis')

Verificamos que quase todos os valores de *AVERAGE_RAIN* são nulos e que existe uma percentagem significativa de valores nulos de *AVERAGE_CLOUDINESS*.

In [17]:
print(df.LUMINOSITY.unique())
print(df.AVERAGE_CLOUDINESS.unique())
print(df.city_name.unique())
print(df.AVERAGE_RAIN.unique())
print(df.AVERAGE_PRECIPITATION.unique())
print(df.record_date.unique())

Verifica-se que existem categorias nas *features* ***AVERAGE_CLOUDINESS*** e ***AVERAGE_PRECIPITATION*** que dizem respeito exatamente à mesma coisa.  

***AVERAGE_CLOUDINESS***
* 'nuvens quebradas' é o mesmo que 'nuvens quebrados'
* 'tempo nublado' é o mesmo que 'nublado'

***AVERAGE_PRECIPITATION***
* 'chuva intensidade pesada' é o mesmo que 'chuva intensidade pesado'
* etc.


In [18]:
for c in df.columns:
    valores_unicos = df[c].unique()
    if len(valores_unicos) < 10:
        print(f'{c} : {valores_unicos}')

Verifica-se que as *features* ***LUMINOSITY***, ***AVERAGE_CLOUDINESS***, ***AVERAGE_RAIN*** e ***AVERAGE_PRECIPITATION***, apesar de possuírem dados categóricos, estes têm uma ordem implícita entre si. 

## Exploração de dados

In [81]:
sea = sns.catplot(data=df, y="AVERAGE_HUMIDITY", x="AVERAGE_CLOUDINESS")
sea.fig.set_figwidth(22)
sea.fig.set_figheight(5)

In [20]:
sns.boxplot(x=df['AVERAGE_TIME_DIFF'])

In [21]:
outliers = df[df['AVERAGE_TIME_DIFF'] > 205]
outliers_indexes = list(outliers.index)
df2 = df.drop(outliers_indexes)

sns.boxplot(x = df2['AVERAGE_TIME_DIFF'])

In [22]:
sns.boxplot(x=df['AVERAGE_TEMPERATURE'])

In [23]:
sns.boxplot(x=df['AVERAGE_FREE_FLOW_SPEED'])

In [24]:
sns.boxplot(x=df['AVERAGE_FREE_FLOW_TIME'])

In [86]:
cols = ['AVERAGE_SPEED_DIFF', 'AVERAGE_FREE_FLOW_SPEED', 'AVERAGE_TIME_DIFF', 'AVERAGE_FREE_FLOW_TIME', 'AVERAGE_TEMPERATURE', 'AVERAGE_ATMOSP_PRESSURE', 'AVERAGE_HUMIDITY', 'AVERAGE_WIND_SPEED']

sns.pairplot(df[cols],hue='AVERAGE_SPEED_DIFF', height=2.5)

## Verificar quais dos valores de cloudiness dizem respeito à mesma coisa

In [91]:
cols = ['AVERAGE_SPEED_DIFF', 'AVERAGE_FREE_FLOW_SPEED', 'AVERAGE_TIME_DIFF', 'AVERAGE_FREE_FLOW_TIME', 'AVERAGE_TEMPERATURE', 'AVERAGE_ATMOSP_PRESSURE', 'AVERAGE_HUMIDITY', 'AVERAGE_WIND_SPEED','AVERAGE_CLOUDINESS']
colunas_a_analisar = df[cols]

df['AVERAGE_CLOUDINESS'] = pd.Categorical(df['AVERAGE_CLOUDINESS'])    
sns.pairplot(colunas_a_analisar[df['AVERAGE_CLOUDINESS'].isin(['céu limpo','céu claro'])],hue='AVERAGE_CLOUDINESS', height=2.5)

In [97]:
df['AVERAGE_CLOUDINESS'] = pd.Categorical(df['AVERAGE_CLOUDINESS'])    
sns.pairplot(colunas_a_analisar[df['AVERAGE_CLOUDINESS'].isin(['nuvens quebradas','nuvens quebrados'])],hue='AVERAGE_CLOUDINESS', height=2.5)

In [99]:
df['AVERAGE_CLOUDINESS'] = pd.Categorical(df['AVERAGE_CLOUDINESS'])    
sns.pairplot(colunas_a_analisar[df['AVERAGE_CLOUDINESS'].isin(['tempo nublado','nublado'])],hue='AVERAGE_CLOUDINESS', height=2.5)

In [105]:
sns.pairplot(colunas_a_analisar,hue='AVERAGE_CLOUDINESS', height=2.5)

In [104]:
bigFig()
sea = sns.histplot(data=df, x="AVERAGE_CLOUDINESS")

In [85]:
aux = df[['AVERAGE_HUMIDITY','AVERAGE_CLOUDINESS']].groupby(by='AVERAGE_CLOUDINESS').mean()
#for col in aux.columns:
#    aux[col] = aux[col].std()
aux

In [44]:
# Visualizar o impacto da luminosity no trânsito
df['AVERAGE_SPEED_DIFF'] = pd.Categorical(df['AVERAGE_SPEED_DIFF'], ["None", "Low", "Medium", "High", "Very_High"])    
    
sea = sns.FacetGrid(df, col = "LUMINOSITY", margin_titles = True)
sea.fig.set_figwidth(12)
sea.fig.set_figheight(5)
sea.map(sns.histplot, 'AVERAGE_SPEED_DIFF')

In [95]:
df.groupby(by='AVERAGE_CLOUDINESS').mean()

## Tratamento de dados

### Transformarção de dados categóricos em ordinais

In [ ]:
def categorical_to_ordinal(df):
    df['LUMINOSITY'].replace({"DARK": 0, "LOW_LIGHT": 1, "LIGHT": 2}, inplace=True)     
    df['AVERAGE_CLOUDINESS'].replace({"céu limpo": 0, "céu claro": 1, "nuvens dispersas": 2,"nuvens quebrados": 3,"nuvens quebradas": 3,"algumas nuvens": 4,"céu pouco nublado":5,'tempo nublado': 6,'nublado': 6}, inplace=True) 
    if 'AVERAGE_SPEED_DIFF' in df:
        df['AVERAGE_SPEED_DIFF'].replace({"None": 0, "Low": 1, "Medium": 2, "High": 3, "Very_High": 4}, inplace=True)
    return df

### Tratamento do campo 'record_date'

In [ ]:
#Identificação de partes do dia (transformação da hora num dado categórico)
def daypart(hour):
    if hour > 0 and hour < 7:
        return "dawn"
    elif hour >= 7 and hour <= 10:
        return "early morning"
    elif hour > 10 and hour < 12:
        return "late morning"
    elif hour >= 12 and hour <= 14:
        return "lunch"
    elif hour > 14 and hour < 17:
        return "early afternoon"
    elif hour >= 17 and hour <= 20:
        return "late afternoon"
    elif hour > 20 and hour < 22:
        return "evening"
    else:
        return "midnight"
    
def is_weekend(day_name):
    if day_name == 'Saturday' or day_name == 'Sunday':
        return 1
    else:
        return 0

def is_friday(day_name):
    if day_name == 'Friday':
        return 1
    else:
        return 0 
    
def season(month):
    if month > 3 and month < 6:
        return "spring"
    elif month >= 6 and month <= 9:
        return "summer"
    elif month > 9 and month < 12:
        return "fall"
    else:
        return "winter"

# Função principal que trata do campo 'record_date'
# Passam a existir colunas que identificam a altura do dia, a estação do ano, bem como se é ou não fim-de-semana/sexta-feira/quinta-feira
def handle_date(df):
    if 'Is_weekend' not in df:
        df.record_date = pd.to_datetime(df.record_date)
        df['Month'] = df.record_date.dt.month
        df['Hour'] = df.record_date.dt.hour
        df['Day_name'] = df.record_date.dt.day_name()
        df['Day_Part'] = df['Hour'].apply(daypart)
        df['Is_weekend'] = df['Day_name'].apply(is_weekend)
        df['Is_friday'] = df['Day_name'].apply(is_friday)
        df['Season'] = df['Month'].apply(season)
        one_hot_dates = pd.get_dummies(df['Day_Part'])
        #one_hot_seasons = pd.get_dummies(df['Season'])
        df = pd.concat([df, one_hot_dates], axis=1)
        df = df.drop(['Month', 'Hour', 'Day_name', 'Day_Part', 'Season', 'record_date'],axis=1)
    return df
        

### Tatamento de *missing values*

In [ ]:
# city_name e AVERAGE_PRECIPITATION só têm um valor único, logo não são relevantes
# Na coluna AVERAGE_RAIN quase todos os valores são nulos 
def drop_columns(df):
    return df.drop(['city_name','AVERAGE_PRECIPITATION','AVERAGE_RAIN'],axis=1)

In [ ]:
def filling_missing_values(df):
    df['AVERAGE_CLOUDINESS'] = df['AVERAGE_CLOUDINESS'].fillna(method='bfill')
    df['AVERAGE_CLOUDINESS'] = df['AVERAGE_CLOUDINESS'].fillna(method='ffill') ???????????????????????
    return df

### Tratamento de *outliers*

In [ ]:
# Tratamento de outliers no campo 'AVERAGE_TIME_DIFF'
def handle_averagetimediff_outliers(df):
    outliers = df[df['AVERAGE_TIME_DIFF'] > 205]
    outliers_indexes = list(outliers.index)
    return df.drop(outliers_indexes)

In [ ]:
def tratamentoDados(filename):
    df = drop_columns(df)
    df = categorical_to_ordinal(df)
    df = handle_date(df)
    df = handle_averagetimediff_outliers(df)
   

def reloadDataset(df):
    df = pd.read_csv(training_file,encoding = "ISO-8859-1")

In [ ]:
df.head()

In [ ]:
df.info()

## **Dados tratados. Fazer predictions**

In [ ]:
x = df_prepared.drop(['AVERAGE_SPEED_DIFF'], axis=1)
y = df_prepared['AVERAGE_SPEED_DIFF'].to_frame()

In [ ]:
# Split for test
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20, random_state=2000)

In [ ]:
clf = DecisionTreeClassifier(random_state = 2021)
scores=cross_val_score(clf,x_train,y_train,cv=10)
print("RESULT: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(),scores.std()))

In [ ]:
label = 'AVERAGE_SPEED_DIFF'
sns.countplot(x=label, data = y_train)

In [ ]:
label = 'AVERAGE_SPEED_DIFF'
sns.countplot(x=label, data = y_test)

In [ ]:
clf.fit(x_train,y_train)
predictions = clf.predict(x_test)
predictions

In [ ]:
print("%0.2f accuracy" % (accuracy_score(y_test,predictions)))

In [ ]:
criterion = ['gini', 'entropy']
max_depth = [2,3,4,6,8,10,12]

param_grid = {'criterion':criterion, 'max_depth':max_depth}
grid = GridSearchCV(DecisionTreeClassifier(random_state=2021),param_grid,refit=True,verbose=3)
grid.fit(x_train, y_train)

In [ ]:
grid_predictions = grid.predict(x_test)
print(classification_report(y_test,grid_predictions))

In [ ]:
grid.best_params_

In [ ]:
grid_predictions

## **Usar o modelo para prever as labels dos dados de teste**

In [ ]:
df_training = tratamentoDados(training_file)
df_test = tratamentoDados(test_file)

In [ ]:
df_training.info()

In [ ]:
print(sns.histplot(data=df_training[df_training['Is_weekend']==1], hue = 'Is_weekend', x = 'AVERAGE_SPEED_DIFF'))

In [ ]:
print(sns.histplot(data=df_training[df_training['Is_weekend']==0], hue = 'Is_weekend', x = 'AVERAGE_SPEED_DIFF'))

In [ ]:
print(sns.histplot(data=df_training[df_training['Is_friday']==0], hue = 'Is_friday', x = 'AVERAGE_SPEED_DIFF'))

In [ ]:
print(sns.histplot(data=df_training[df_training['Is_friday']==1], hue = 'Is_friday', x = 'AVERAGE_SPEED_DIFF'))

In [ ]:
print(sns.histplot(data=df_training[df_training['dawn']==1], x = 'AVERAGE_SPEED_DIFF'))

In [ ]:
print(sns.histplot(data=df_training[df_training['early morning']==1], x = 'AVERAGE_SPEED_DIFF'))

In [ ]:
print(sns.histplot(data=df_training[df_training['late morning']==1], x = 'AVERAGE_SPEED_DIFF'))

In [ ]:
print(sns.histplot(data=df_training[df_training['lunch']==1], x = 'AVERAGE_SPEED_DIFF'))

In [ ]:
print(sns.histplot(data=df_training[df_training['early afternoon']==1], x = 'AVERAGE_SPEED_DIFF'))

In [ ]:
print(sns.histplot(data=df_training[df_training['late afternoon']==1], x = 'AVERAGE_SPEED_DIFF'))

In [ ]:
print(sns.histplot(data=df_training[df_training['evening']==1], x = 'AVERAGE_SPEED_DIFF'))

In [ ]:
print(sns.histplot(data=df_training[df_training['midnight']==1], x = 'AVERAGE_SPEED_DIFF'))

In [ ]:
x = df_training.drop(['AVERAGE_SPEED_DIFF'], axis=1)
y = df_training['AVERAGE_SPEED_DIFF'].to_frame()

In [ ]:
criterion = ['gini', 'entropy']
max_depth = [2,3,4,6,8,10,12]

param_grid = {'criterion':criterion, 'max_depth':max_depth}

grid = GridSearchCV(DecisionTreeClassifier(random_state=2021),param_grid,refit=True,verbose=3)
grid.fit(x, y)
grid_predictions = grid.predict(df_test)

In [ ]:
grid_predictions

In [ ]:
tryMane = pd.DataFrame(data=grid_predictions, index=None, columns=['Speed_Diff'])
tryMane

In [ ]:
tryMane['RowId'] = range(1,1501)
tryMane['Speed_Diff'].replace({0:"None", 1:"Low", 2:"Medium", 3:"High", 4:"Very_High"}, inplace=True)
tryMane = tryMane[['RowId','Speed_Diff']] # Inverter a ordem das colunas para concordar com o formato desejado
tryMane

In [ ]:
try_nr = 4
outputfile = f'mane_try{try_nr}.csv'

tryMane.to_csv(outputfile,index=False)
submission = pd.read_csv(outputfile,encoding = "ISO-8859-1")
submission.head()